# Vectorial Word Representations

## Background
Representing words as dense vectors over a finite-dimensional space was one of the recent breakthroughs in Natural Language Processing. Vectorial representations allow space-efficient, informationally rich storage of words that adequately captures their semantic content and enables numerical computation on them. Word vectors are the standard input representation for language-oriented machine learning architectures. Even though new methods for constructing such representations emerge frequently, the original set of published papers remain a de facto point of reference as well as a good starting point. For this assignment, you will be asked to implement a small-scale variant of one such paper, namely [Global Word Vectors for Word Representation](https://nlp.stanford.edu/pubs/glove.pdf).

Much of the code and data pre-processing has already been done for you. Additionally, notes on the paper will appear throughout the notebook to guide you along the code. It is, however, important to read and understand the paper, its terminology and the theory behind it before attempting to go through with the assignment.

For conceptual questions, give short but descriptive answers.

## Corpus Statistics

The paper's proposed model, GloVe, aims to densely represent words in a way that captures the global corpus statistics. 

The construction it encodes is the word __co-occurrence matrix__. A co-occurrence matrix is a simplistic data structure that counts the amount of times each word has appeared within the context of each other word. The definition of a context varies between implementations; usually, context is implied to be a fixed-length span (that may or may not be allowed to escape sentence boundaries) around a word. 

For instance, in the sentence below and for a context length of 2, the word <span style="color:pink">__Earth__</span> occurs in the context of <span style="color:lightgreen">made</span> (1), <span style="color:lightgreen">on</span> (1), <span style="color:lightgreen">as</span> (1), <span style="color:lightgreen">an</span> (1).

> "He struck most of the friends he had <span style="color:lightgreen">made on</span> <span style="color:pink">__Earth__</span> <span style="color:lightgreen">as an</span> eccentric"

Similarly, the word <span style="color:pink">__friends__</span> occurs in the context of <span style="color:lightgreen">of</span> (1), <span style="color:lightgreen">the</span> (1), <span style="color:lightgreen">he</span> (1), <span style="color:lightgreen">had</span> (1).

> "He struck most <span style="color:lightgreen">of the</span> <span style="color:pink">__friends__</span> <span style="color:lightgreen">he had</span> made on Earth as an eccentric"

An alternative definition of a context would be, for instance, the variable-length windows spanned by a full sentence.

Contexts may be summed across sentences or entire corpora; the summed context of <span style="color:pink">he</span> in the example sentence is: <span style="color:lightgreen">struck</span> (1), <span style="color:lightgreen">most</span> (1), <span style="color:lightgreen">the</span> (1), <span style="color:lightgreen">friends</span> (1), <span style="color:lightgreen">had</span> (1), <span style="color:lightgreen">made</span> (1).



For the purposes of this assignment, we have prepared a co-occurrence matrix over a minimally processed version of the Harry Potter books. The pickle file contains three items:
1. `vocab`: a dictionary mapping words to unique ids, containing $N$ unique words
1. `contexts`: a dictionary mapping words to their contexts, where contexts are themselves dicts from words to ints
2. `X`: a torch LongTensor $\mathbf{X}$ of size $N \times N$, where $\mathbf{X}[i,j]$ denotes the number of times the word with id $j$ has appeared in the context of the word with id $i$

Extremely common or uncommon words (i.e. words with too few or too many global occurrences) have been filtered out for practical reasons.

In [1]:
import pickle
import torch
from torch import FloatTensor, LongTensor
from typing import Dict, Callable, List

In [2]:
with open('output.p', 'rb') as f:
    vocab, contexts, X = pickle.load(f)
print(len(vocab))

5359


Let's inspect the summed context of the word 'portrait'.

In [3]:
sorted([(item, value) for item, value in contexts['portrait'].items()], key=lambda x: x[1], reverse=True)

[('harry', 103),
 ('said', 97),
 ('hole', 84),
 ('ron', 49),
 ('hermione', 46),
 ('room', 41),
 ('t', 40),
 ('fat', 39),
 ('lady', 39),
 ('common', 30),
 ('dumbledore', 26),
 ('phineas', 24),
 ('climbed', 22),
 ('just', 19),
 ('swung', 17),
 ('sirius', 17),
 ('professor', 17),
 ('time', 16),
 ('voice', 15),
 ('open', 15),
 ('got', 15),
 ('nigellus', 15),
 ('reached', 14),
 ('like', 14),
 ('came', 14),
 ('little', 14),
 ('gryffindor', 13),
 ('turned', 13),
 ('forward', 12),
 ('don', 12),
 ('long', 12),
 ('place', 12),
 ('wall', 11),
 ('neville', 11),
 ('black', 11),
 ('going', 11),
 ('snape', 11),
 ('hall', 11),
 ('mcgonagall', 11),
 ('corridor', 10),
 ('walked', 10),
 ('away', 10),
 ('ve', 10),
 ('way', 10),
 ('visit', 10),
 ('good', 10),
 ('did', 10),
 ('look', 10),
 ('password', 9),
 ('moment', 9),
 ('know', 9),
 ('sir', 9),
 ('opened', 9),
 ('face', 9),
 ('heard', 9),
 ('come', 8),
 ('gone', 8),
 ('asked', 8),
 ('let', 8),
 ('really', 8),
 ('entrance', 8),
 ('cadogan', 8),
 ('mother

How about the word 'ghost'?

In [4]:
sorted([(item, value) for item, value in contexts['ghost'].items()], key=lambda x: x[1], reverse=True)

[('harry', 52),
 ('said', 36),
 ('nick', 22),
 ('t', 20),
 ('nearly', 20),
 ('headless', 18),
 ('know', 15),
 ('looked', 12),
 ('ron', 10),
 ('saw', 9),
 ('d', 9),
 ('ve', 9),
 ('got', 8),
 ('gryffindor', 8),
 ('years', 7),
 ('bloody', 7),
 ('baron', 7),
 ('wand', 7),
 ('cedric', 7),
 ('dumbledore', 7),
 ('just', 6),
 ('don', 6),
 ('think', 6),
 ('told', 6),
 ('slytherin', 6),
 ('staring', 6),
 ('eyes', 6),
 ('professor', 6),
 ('passed', 6),
 ('like', 6),
 ('hermione', 6),
 ('gray', 6),
 ('ruff', 5),
 ('suddenly', 5),
 ('table', 5),
 ('potter', 5),
 ('giving', 5),
 ('began', 5),
 ('tower', 5),
 ('magic', 5),
 ('binns', 5),
 ('light', 5),
 ('held', 5),
 ('head', 5),
 ('little', 5),
 ('solid', 5),
 ('really', 4),
 ('hufflepuff', 4),
 ('hat', 4),
 ('opposite', 4),
 ('seen', 4),
 ('arm', 4),
 ('need', 4),
 ('sir', 4),
 ('gaunt', 4),
 ('face', 4),
 ('silver', 4),
 ('corridor', 4),
 ('covered', 4),
 ('wide', 4),
 ('oh', 4),
 ('asked', 4),
 ('away', 4),
 ('myrtle', 4),
 ('hair', 4),
 ('thing'

The co-occurrence matrix of a very large corpus should give a meaningful summary of how a word is used in general. A single row of that matrix is already a __word vector__ of size $N$. However such vectors are extremely sparse, and for large corpora the size of $N$ will become unwieldy. We will follow along the paper in designing a neural algorithm that can compress the word vectors while retaining most of their informational content. 

<div class="alert alert-block alert-warning">
<b>Note:</b>
For the resulting vectors to actually be informative, the source corpus should have a size of at least a few billion words; on the contrary, our corpus enumerates merely a million words, so we can't expect our results to be as great.
</div>

### Assignment 1.0:  Sparsity and Stability

Our matrix $\mathbf{X}$ is very sparse; most of its elements are zero. Find what the ratio of non-zero elements is.

_Hint_: The function `non_zero_ratio` should return a `float` rather than a `FloatTensor`. Remember `.item()`.

We will soon need to perform division and find the logarithm of $\mathbf{X}$. Neither of the two operations are well-defined for $0$.

Change the matrix's datatype to a `torch.float` and add a small constant to it (e.g. $0.1$) to ensure numerical stability while maintaining sparsity.

In [5]:
def non_zero_ratio(sparse_matrix: LongTensor) -> float:
    non_zero_el = torch.nonzero(sparse_matrix).size(0) # nonzero returns a tensor with all the indexes of non-zero elements
    tot_el = torch.numel(sparse_matrix)
    return non_zero_el/tot_el

In [6]:
print(non_zero_ratio(X))
X = X.float()
X = X + 0.1

0.108448515107535


### Assignment 1.1: From co-occurrence counts to probabilities
From the paper: 
> Let the matrix of word-word co-occurrence counts be denoted by $X$, whose entries $X_{ij}$ tabulate the number of times word $j$ occurs in the context of word $i$.  Let $X_i$= $\sum_{k} X_{ik}$ be the number of times any word appears in the context of word $i$. Finally, let $P_{ij} = P(j  | i) =  X_{ij}/X_i$ be the probability that word $j$ appear in the context of word $i$. 

Complete the function `to_probabilities`, that accepts a co-occurrence matrix and returns the probability matrix $P$. 

_Hint_: Remember broadcasting and `torch.sum()`.

In [7]:
def to_probabilities(count_matrix: FloatTensor) -> FloatTensor:
    divider = torch.sum(count_matrix, dim=1)
    divider = divider.view(divider.shape[0], 1)
    return count_matrix/divider

P = to_probabilities(X)

### Assignment 1.2: Probing words

From the paper:
> Consider two words $i$ and $j$ that exhibit a particular aspect of interest. The relationship of these words can be examined by studying the ratio of their co-occurrence probabilities with various probe words, $k$.  For words $k$ related to $i$ but not $j$, we expect the ratio $P_{ik}/P_{jk}$ will be large.  Similarly, for words $k$ related to $j$ but not $i$, the ratio should be small. For words $k$ that are either related to both $i$ and $j$, or to neither, the ratio should be close to one.

Complete the function `query` that accepts two words $w_i$ and $w_j$, a vocab $V$ and a probability matrix $\mathbf{P}$, maps each word to its corresponding index and returns the probability $P(j  |  i)$.

Then, complete the function `probe` that accepts three words $w_i$, $w_j$ and $w_k$, a vocab $V$ and a probability matrix $\mathbf{P}$, calls `query` and returns the ratio $P(k |  i) / P(k  |  j)$.

In [8]:
def query(word_i: str, word_j: str, vocab: Dict[str, int], probability_matrix: FloatTensor) -> float:  
    i = vocab[word_i]
    j = vocab[word_j]
    return probability_matrix[i,j].item()

def probe(word_i: str, word_j: str, word_k: str, vocab: Dict[str, int], probability_matrix: FloatTensor) -> float:
    return query(word_i, word_k, vocab, probability_matrix)/query(word_j, word_k, vocab, probability_matrix)

Let's probe a few words and examine whether the authors' claim holds even for our (tiny) corpus. 

In [9]:
print(probe('tea', 'wand', 'spell', vocab, P))
print(probe('tea', 'wand', 'cup', vocab, P))

print(probe('voldemort', 'hagrid', 'curse', vocab, P))
print(probe('voldemort', 'hagrid', 'beast', vocab, P))

print(probe('mcgonagall', 'snape', 'potions', vocab, P))
print(probe('mcgonagall', 'snape', 'transfiguration', vocab, P))

print(probe('hedwig', 'scabbers', 'owl', vocab, P))
print(probe('hedwig', 'scabbers', 'rat', vocab, P))

print(probe('ron', 'hermione', 'book', vocab, P))
print(probe('ron', 'hermione', 'red', vocab, P))

0.42263377158243687
24.61305209040928
8.710822994956738
0.5588329712750296
0.0036196233324942884
43.71879996208764
5.626578741777974
0.017819201079668088
0.678307074092271
2.0382544087903236


So it does seem like we are getting sensible results for in-domain words. Of course, probing out-of-domain words (such as the thermodynamics example the authors present) does not go all that well.. 

In [10]:
print(probe('ice', 'steam', 'solid', vocab, P))
print(probe('ice', 'steam', 'gas', vocab, P))

0.08204095852267408
0.9024505003445088


(Take home message: HP books are probably not the best textbook on thermodynamics)

## Dense Vectors

Now, we would like to convert these long, spare vectors into short, dense ones. 

The conversion should be such that the probability ratios we inspected earlier may still be reconstructed via some (for now, unknown) operation $F$ on the dense vectors.

To restrict the search space over potential functions, the authors impose a number of constraints they think $F$ should satisfy:
1. > While $F$ could be taken to be a complicated function parameterized by, e.g., a neural network, doing so would obfuscate the linear structure we are trying to capture. $F$ should be dot-product based.
2. > The distinction between a word and a context word is arbitrary and we are free to exchange the two roles. To do so consistently, we must not only exchange $w \leftrightarrow \tilde{w}$ but also $X \leftrightarrow X^T$.
3. > It should be well-defined for all values in $X$.

Given these three constraints, each word $i$ of our vocabulary is represented by four vectors:
1. A vector $w_i \in \mathbb{R}^D$
2. A bias $b_i \in \mathbb{R}$
3. A context vector $\tilde{w}_i \in \mathbb{R}^D$
4. A context bias $\tilde{b}_i \in \mathbb{R}$

and $F: \mathbb{R}^D \times \mathbb{R} \times \mathbb{R}^D \times \mathbb{R} \to \mathbb{R}$ is defined as:

$F(w_i, \tilde{w}_j, b_i, \tilde{b}_k) = w_i^T\tilde{w}_k + b_i + \tilde{b}_k$

such that $F(w_i, \tilde{w}_k, b_i, \tilde{b}_k)$ approximates $log(\mathbf{X}_{ik})$, 

or equivallently the least squares error $J$ is minimized, where:

$J = \sum_{i,j=1}^{V} f(X_{ij})(w_{i}^T\tilde{w}_j + b_i + \tilde{b}_j - log(X_{ij}))^2$ the loss term

and 

$f: \mathbb{R} \to \mathbb{R} = \begin{cases}
    (x/x_{max})^a, & \text{if $x<x_{max}$}\\
    1, & \text{otherwise}.
  \end{cases}$ a term weighting function 

### Assignment 1.3: Weighting Function

Let's start with the last part. Complete the weighting function `weight_fn` which accepts a co-occurrence matrix $\mathbf{X}$, a maximum value $x_{max}$ and a fractional power $a$, and returns the weighted co-occurrence matrix $f(\mathbf{X})$.

Then, compute `X_weighted`, the weighting of $\mathbf{X}$, using the paper's suggested parameters.

_Hint_: Note that $f$ is defined point-wise, so our weighting function should also be point-wise.

In [11]:
def weight_fn(X: FloatTensor, x_max: int, a: float) -> FloatTensor:
    return torch.where(X<x_max, (X/x_max)**a, torch.ones(X.shape))

X_weighted = weight_fn(X, 100, 0.75)

### Assignment 1.4: Loss Function

Next step is to write the loss function. 

We can write it as a point-wise function, apply it iteratively over each pair of words and then sum the result; that's however extremely inefficient. 

Inspecting the formulation of $J$, it is fairly straight-forward to see that it can be immediately implemented using matrix-matrix operations, as:

$J = \sum_{i,j=1}^{V}f(\mathbf{X})\cdot(W\tilde{W}^T + b + \tilde{b}^T - log(X))^2$,

where $W$, $\tilde{W}$ the $N \times D$ matrices containing the $D$-dimensional vectors of all our $N$ vocabulary words, and $b$, $\tilde{b}$ the $N \times 1$ matrices containing the $1$-dimensional biases of our words.

Complete `loss_fn`, a function that accepts a weighted co-occurrence matrix $f(\mathbf{X})$, the word vectors and biases $W$, $\tilde{W}$, $b$, $\tilde{b}$ and the co-occurrence matrix $\mathbf{X}$, and computes $J$. What is the expected shape of $J$ before and after the summation?

In [12]:
def loss_fn(X_weighted: FloatTensor, W: FloatTensor, W_context: FloatTensor, 
            B: FloatTensor, B_context: FloatTensor, X: FloatTensor) -> FloatTensor:
    loss = torch.sum(X_weighted * (W @ W_context.t() + B + B_context.t() - torch.log(X)) ** 2)
    return loss


### Assignment 1.5: GloVe

We have the normalized co-occurrence matrix $\mathbf{X}$, the weighting function $f$, and the loss function $J$ that implements $F$.

What we need now is a mapping from words (or word ids) to unique, parametric and trainable vectors. 

Torch provides this abstraction in the form of [Embedding layers](https://pytorch.org/docs/stable/nn.html#embedding). Each such layer may be viewed as a stand-alone network, that can be optimized using the standard procedure we have already seen. 

We will utilize the `nn.Module` class to contain all our embedding layers and steamline their joint optimization.
The container class will be responsible for a few things:

1. Wrapping the embedding layers:
    1. A vector embedding that maps words to $w \in \mathbb{R}^D$
    1. A context vector embedding that maps words to $w_c \in \mathbb{R}^D$
    1. A bias embedding that maps words to $b \in \mathbb{R}^1$
    1. A context bias embedding that maps words to $b_c \in \mathbb{R}^1$
1. Implementing `forward`, a function that accepts a weighted co-occurrence matrix $f(\mathbf{X})$, the co-occurrence matrix $\mathbf{X}$, then computes the embeddings of all words and finally calls `loss_fn` as defined above.
1. Implementing `get_vectors`, a function that receives no input and produces the word vectors and context word vectors of all words, adds them together and returns the result, in accordance with the paper:
> .. With this in mind, we choose to use the sum $W + \tilde{W}$ as our word vectors.

Complete the network class following the above specifications.


In [13]:
class GloVe(torch.nn.Module):
    def __init__(self, vocab: Dict[str, int], vector_dimensionality: int=30, device: str='cpu') -> None:
        super(GloVe, self).__init__()
        self.device = device
        self.vocab_len = len(vocab)
        self.w = torch.nn.Embedding(num_embeddings = self.vocab_len, embedding_dim=vector_dimensionality).to(self.device)
        self.wc = torch.nn.Embedding(num_embeddings = self.vocab_len, embedding_dim=vector_dimensionality).to(self.device)
        self.b = torch.nn.Embedding(num_embeddings = self.vocab_len, embedding_dim=1).to(self.device)
        self.bc = torch.nn.Embedding(num_embeddings = self.vocab_len, embedding_dim=1).to(self.device)
        
    def forward(self, X_weighted: FloatTensor, X: FloatTensor) -> FloatTensor:
        embedding_input = torch.arange(self.vocab_len).to(self.device)
        W = self.w(embedding_input)
        WC = self.wc(embedding_input)
        B = self.b(embedding_input)
        BC = self.bc(embedding_input)       
        return loss_fn(X_weighted, W, WC, B, BC, X)
    
    def get_vectors(self) -> FloatTensor:
        embedding_input = torch.arange(self.vocab_len).to(self.device)
        W = self.w(embedding_input)
        WC = self.wc(embedding_input)
        return W + WC

### Assignment 1.6: Training

Everything is in place; now we may begin optimizing our embedding layers (and in doing so, the vectors they assign). Instantiate the network class you just defined, using $D = 30$. Then instantiate an `Adam` optimizer with a learning rate of 0.05 and train your network for 300 epochs.

When writing the training script, remember that your network's forward pass is __already__ computing the loss.

In [14]:
network = GloVe(vocab)
opt = torch.optim.Adam(network.parameters(), lr=0.05)

In [15]:
num_epochs = 300

for i in range(num_epochs):
    loss = network.forward(X_weighted, X)
    loss.backward()
    opt.step() 
    opt.zero_grad() 
    if i%10 == 0:
        print('Iteration {} loss: {}'.format(i, loss.item()))


Iteration 0 loss: 11838599.0
Iteration 10 loss: 3171371.5
Iteration 20 loss: 1178918.75
Iteration 30 loss: 565795.375
Iteration 40 loss: 411583.03125
Iteration 50 loss: 356158.3125
Iteration 60 loss: 332442.75
Iteration 70 loss: 320096.34375
Iteration 80 loss: 311698.875
Iteration 90 loss: 304708.25
Iteration 100 loss: 298670.8125
Iteration 110 loss: 293599.875
Iteration 120 loss: 289392.5
Iteration 130 loss: 285875.1875
Iteration 140 loss: 282890.8125
Iteration 150 loss: 280318.375
Iteration 160 loss: 278071.8125
Iteration 170 loss: 276092.4375
Iteration 180 loss: 274337.75
Iteration 190 loss: 272774.9375
Iteration 200 loss: 271379.125
Iteration 210 loss: 270131.625
Iteration 220 loss: 269015.3125
Iteration 230 loss: 268013.875
Iteration 240 loss: 267113.09375
Iteration 250 loss: 266299.4375
Iteration 260 loss: 265560.5625
Iteration 270 loss: 264884.5
Iteration 280 loss: 264261.6875
Iteration 290 loss: 263683.875


### Assignment 1.7: Validation (Similarity)

Curious to see what this network has learned? Let's perform a simple validation experiment. 

We will check which words the models considers the most similar to other words. To that end, we need a notion of __similarity__. One of the most common measures of similarity in high dimensional vector spaces is the cosine similarity. 

The cosine similarity of two vectors $\vec{a}, \vec{b}$ is given as:
$$sim(\vec{a}, \vec{b}) = \frac{\vec{a}\cdot \vec{b}}{|\vec{a}|_2 \cdot |\vec{b}|_2}$$

where $|\vec{x}|$_2 the $L_2$-norm of the $\vec{x}$.

The function `similarity` below accepts two words, a vocabulary and the network's output vectors, and computes the similarity between these two words.

In [16]:
def similarity(word_i: str, word_j: str, vocab: Dict[str, int], vectors: FloatTensor) -> float:
    i = vocab[word_i]
    j = vocab[word_j] 
    v_i = vectors[i] / torch.norm(vectors[i], p=2)  # a/|a|
    v_j = vectors[j] / torch.norm(vectors[j], p=2)  # b/|b|
    sim = torch.mm(v_i.view(1, -1), v_j.view(-1, 1)).item()
    return sim

Let's see some examples (try your own word pairs):

In [17]:
word_vectors = network.get_vectors().detach()

for pair in [('cruciatus', 'imperius'), 
             ('avada', 'kedavra'), 
             ('hogwarts', 'school'), 
             ('goblin', 'hagrid'), 
             ('giant', 'hagrid'),
             ('avada', 'hermione'), 
             ('avada', 'voldemort'), 
             ('hand', 'wand'),
             ('quidditch', 'dumbledore'),
             ('professor', 'snape'),
             ('professor', 'petunia'),
             ('professor', 'cup'),
            ]:
    
    print('\nSimilarity between {} and {} is: {}'.
          format(pair[0], pair[1], similarity(pair[0], pair[1], vocab, word_vectors)))


Similarity between cruciatus and imperius is: 0.5489968657493591

Similarity between avada and kedavra is: 0.8792098760604858

Similarity between hogwarts and school is: 0.5746870040893555

Similarity between goblin and hagrid is: 0.19663673639297485

Similarity between giant and hagrid is: 0.5904597043991089

Similarity between avada and hermione is: -0.12604019045829773

Similarity between avada and voldemort is: 0.5594943165779114

Similarity between hand and wand is: 0.5754926204681396

Similarity between quidditch and dumbledore is: 0.16698962450027466

Similarity between professor and snape is: 0.5495306849479675

Similarity between professor and petunia is: 0.10922577977180481

Similarity between professor and cup is: -0.09921182692050934


To obtain the similarities of one word against all other words in the corpus, we may rewrite the above equation as:
$$sim(\vec{w}, \mathbf{C}) = \frac{\vec{w}\cdot \mathbf{C}}{|\vec{w}|_2 \cdot |\mathbf{C}|_2}$$

Using `similarity` as a reference, write `similarities`, which accepts one word, a vocabulary and the network's output vectors and computes the similarity between the word and the entire corpus.

_Hint_: $\mathbf{C} \in \mathbb{R}^{N, D}$, $\vec{w} \in \mathbb{R}^{1, D}$, $sim(\vec{w}, \mathbf{C}) \in \mathbb{R}^{1, N}$

In [18]:
def similarities(word_i: str, vocab: Dict[str, int], vectors: FloatTensor) -> FloatTensor:
    i = vocab[word_i]
    v_i = vectors[i] / torch.norm(vectors[i], p=2)  # a/|a|
    v_C = vectors / torch.norm(vectors, p=2)  # b/|b|
    sims = (torch.mm(v_C, v_i.view(-1, 1))).t()
    return sims

Now we can manipulate the word vectors to find out what the corpus-wide most similar words to a query word is!

In [19]:
def most_similar(word_i: str, vocab: Dict[str, int], vectors: FloatTensor, k: int) -> List[str]:
    sims = similarities(word_i, vocab, vectors)
    _, topi = sims.topk(dim=-1, k=k)
    topi = topi.view(-1).cpu().numpy().tolist()
    inv = {v: i for i, v in vocab.items()}
    return [inv[i] for i in topi if inv[i] != word_i]

In [20]:
for word in ['forbidden', 'myrtle', 'gryffindor', 'wand', 'quidditch', 'marauder', 'horcrux', 'phoenix', 'triwizard', 'screaming',
            'letter'
            ]:
    print('Most similar words to {}: {}'.format(word, most_similar(word, vocab, word_vectors, 6)))

Most similar words to forbidden: ['cabin', 'forest', 'grounds', 'path', 'visit']
Most similar words to myrtle: ['moaning', 'bathroom', 'toilet', 'dobby', 'funny']
Most similar words to gryffindor: ['slytherin', 'quaffle', 'angelina', 'ravenclaw', 'lee']
Most similar words to wand: ['dumbledore', 'just', 'said', 'looked', 'right']
Most similar words to quidditch: ['team', 'wood', 'match', 'seeker', 'snitch']
Most similar words to marauder: ['map', 'parchment', 'tapestry', 'tapped', 'padfoot']
Most similar words to horcrux: ['locket', 'diary', 'sword', 't', 'soul']
Most similar words to phoenix: ['fawkes', 'wand', 'ollivander', 'cornelius', 'feather']
Most similar words to triwizard: ['tournament', 'champions', 'diggory', 'task', 'krum']
Most similar words to screaming: ['pain', 'blinding', 'screamed', 'upward', 'wand']
Most similar words to letter: ['owl', 'hedwig', 'pigwidgeon', 'owls', 'prophet']


Quite impressive; we managed to encode a meaningful portion of the corpus statistics in such $30$ numbers in each word! 
(A compression ratio of 99.4%)

<div class="alert alert-block alert-warning">
<b>Note:</b> The word vectors obtained by this process are (to a small extent) random, due to the random initialization of the embedding layers. If you are unhappy with your results, you can repeat the experiment a few times or try to toy around with the hyper-parameters (the smoothing factor of $\mathbf{X}$, $x_{max}$, $a$, the number of epochs and the dimensionality of the vector space).
</div>

Word vectors, however, contain way more information than just word co-occurrence statistics. Hold tight until the next assignment, where we will see how word vectors may be used to infer information spanning entire phrases and sentences.

### Assignment 1.8: Shortcomings
Evidently, GloVe offers a simple and computationally efficient means to construct dense word representations.
However, the means of vectorization suffers from a few important shortcomings.
Can you imagine what these are? Briefly report on at least two of them.

The first shortcoming is that, through the process of constructing short dense word representations, some information is inevitably lost.

Additionally, the dense, short vectors representing words do not carry explicit information about the words anymore, as did the co-occurrence matrix.
The 30 dimensions of the embedding obtained with this means of vectorization do not represent anything understandable by a human observer. An obsverver could have interpreted each dimension of the long sparse vectors as the co-occurrence of a word with another word.

There are also general shortcomings regarding the adopted approach.
One of these is the inability at learning the representation of words that are not present in the vocabulary.
This leads GolVe to perform poorly in contexts it has not been trained on.
Moreover, words with opposite meaning are usually represented by similar vectors.
There are also some shortcomings regarding computational efficiency: The co-occurrence matrix takes a lot of memory for storage, and when it is necessary to modify the hyper-parameter related to it, the whole matrix must be reconstructed again, which for high dimensionality is very time consuming.

### Assignment 1.9: Validation (Word Analogies)

From the paper:
> The word analogy task consists of questions like "_a_ is to _b_ as is _c_ to ?" To correctly answer this question, we must find the word d such that $w_d \approx w_b - w_a + w_c$ according to the cosine similarity.

Write your own function that performs the word analogy task.

_Hint_: Take a look at the code a few cells back. Most of what you need is already there.

In [21]:
def analogy(word_a: str, word_b: str, word_c: str, vocab: Dict[str, int], vectors: FloatTensor, k: int) -> List[str]:
    a = vocab[word_a]
    b = vocab[word_b]
    c = vocab[word_c]
    dt = vectors[b] - vectors[a] + vectors[c]
    
    v_dt = dt / torch.norm(dt, p=2)
    
    v_C = vectors / torch.norm(vectors, p=2)
    sims = (torch.mm(v_C, v_dt.view(-1, 1))).t()
    
    _, topi = sims.topk(dim=-1, k=k)
    topi = topi.view(-1).cpu().numpy().tolist()
    
    inv = {v: i for i, v in vocab.items()}
    analogies = [inv[i] for i in topi if inv[i] != word_a and inv[i] != word_b and inv[i] != word_c]  
    return analogies[0:6]


Some example triplets to test your analogies on.

In [22]:
triplets = [('padma', 'parvati', 'fred'),
            ('avada', 'kedavra', 'expecto'),
            ('dungeon', 'slytherin', 'tower'),
            ('scabbers', 'ron', 'hedwig'),
            ('ron', 'molly', 'draco'),
            ('durmstrang', 'viktor', 'beauxbatons'),
            ('snape', 'potions', 'trelawney'),
            ('harry', 'seeker', 'ron'),
            ('malfoy', 'slytherin', 'potter'),
            ('headless', 'nick', 'moaning'),
            ('spiders', 'ron', 'dementors'),
            ('buckbeak', 'hippogriff', 'hedwig'),
           ]

for a, b, c in triplets:
    print('{} is to {} as {} is to {}'.format(a, b, c, analogy(a, b, c, vocab, word_vectors, 10)))

padma is to parvati as fred is to ['t', 'said', 'ron', 'weasley', 'd', 'dean']
avada is to kedavra as expecto is to ['patronum', 'stag', 'dementor', 'gregorovitch', 'wand', 'wormtail']
dungeon is to slytherin as tower is to ['gryffindor', 'hufflepuff', 'wood', 'ravenclaw', 'alicia', 'quaffle']
scabbers is to ron as hedwig is to ['t', 'said', 'pigwidgeon', 'harry', 'breakfast', 'd']
ron is to molly as draco is to ['department', 'crouch', 'evans', 'lucius', 'greyback', 'police']
durmstrang is to viktor as beauxbatons is to ['ludo', 'crouch', 'krum', 'blonde', 'madame', 'cho']
snape is to potions as trelawney is to ['divination', 'practical', 'transfiguration', 'pages', 'essay', 'arts']
harry is to seeker as ron is to ['snitch', 'team', 'match', 'flint', 'game', 'warrington']
malfoy is to slytherin as potter is to ['sorting', 'quaffle', 'gryffindor', 'hufflepuff', 'ravenclaw', 'related']
headless is to nick as moaning is to ['bathroom', 'myrtle', 'borgin', 'goyle', 'peeves', 'willow']
spi

Some minimal emergent intelligence :) *(hopefully..)*

🧙‍♀️

### Optional[Extra]
How does training and hyper-parameter choice affect the model's performance?
Repeat the training using your own hyper-parameters (vector space dimensionality, optimizer parameters, training epochs, etc.). 

During the training loop, print the qualitative benchmarks every few epochs. Do they keep improving? Is there any disadvantage to exhaustively training until convergence?